## Team
- 10644931 Ansaldi Jacopo
- XXXXXXXX Omar Emara

## Preprocessing
We use a ... and then ... and then ...

First, we load the libraries we need

In [ ]:
import pandas as pd
pd.options.display.max_columns = 100
import numpy as np

# import matplotlib and allow it to plot inline
import matplotlib.pyplot as plt


# seaborn can generate several warnings, we ignore them
import warnings

warnings.filterwarnings("ignore")

load the datasets and inspect 
speed dataset is loaded by chunks because it is to big to be processed in one block

In [ ]:
def data_exploration(dataset):
    print(dataset.info())
    print(dataset.isnull().sum())
    print(dataset.head(100))

SPEED_TRAIN_PATH = "C:\\Users\erica\Desktop\jacopo\progetto dmtm\speeds_train.csv.gz"
SENSORS_PATH = "C:\\Users\erica\Desktop\jacopo\progetto dmtm\sensors.csv.gz"
EVENT_TRAIN_PATH = "C:\\Users\erica\Desktop\jacopo\progetto dmtm\events_train.csv.gz"

speed_chunks = pd.read_csv(SPEED_TRAIN_PATH,
                           chunksize=150000)  # Specifying chunk size for the speed dataset to be 150,000 rows
sensors = pd.read_csv(SENSORS_PATH)
events = pd.read_csv(EVENT_TRAIN_PATH)

data_exploration(events)
data_exploration(sensors)

events preprocessing:
    1-casting dates
    2-downcasting some values in order to occupy less memory
    3-check missing values and deal with them
    4-rename some columns to avoid duplicates with other datasets

In [ ]:
def check_missing_values(dt):
    print(dt.isnull().sum())

events['START_DATETIME_UTC'] = pd.to_datetime(events['START_DATETIME_UTC'])
events['END_DATETIME_UTC'] = pd.to_datetime(events['END_DATETIME_UTC'])
events['EVENT_DETAIL'] = events['EVENT_DETAIL'].astype(np.int32)
events['KEY'] = events['KEY'].astype(np.int32)
events['KM_END'] = events['KM_END'].astype(np.int32)
events['KM_START'] = events['KM_START'].astype(np.int32)
check_missing_values(events)
#==> 24 missing values on event detail
#since the percentage of missing values is really small(<0.1%) ==> delete all the rows with mv
events = events.dropna()

# Avoiding duplicate columns
events.rename(columns={'KEY': 'KEY_EVENTS', 'KEY_2': 'KEY_2_EVENTS'}, inplace=True)

then for each speed dataset chunk we:
    1)downcasted the values to occupy less memory
    2)merge the chunk with the sensors dataset on the attributes "key" and "km" that identifies the sensor
    3)merge the resulting dataframe with the events dataset basing on the keys, the km range and the time range scaled to each 
    each quarter hour like in the speed dataset. So we obtained a dataset in which for every quarter hour there are 4 columns
    that specify if there is one or more events occuring (4 because is the max num of simultaneous events)

In [ ]:
def down_casting_data_types(speeds_chunk):
    # Down-casting the data types of some columns to 32-bit in order to reduce memory usage
    speeds_chunk['KM'] = speeds_chunk['KM'].astype(np.int32)
    speeds_chunk['KEY'] = speeds_chunk['KEY'].astype(np.int32)
    speeds_chunk['SPEED_AVG'] = speeds_chunk['SPEED_AVG'].astype(np.float32)
    speeds_chunk['SPEED_SD'] = speeds_chunk['SPEED_SD'].astype(np.float32)
    speeds_chunk['SPEED_MIN'] = speeds_chunk['SPEED_MIN'].astype(np.float32)
    speeds_chunk['SPEED_MAX'] = speeds_chunk['SPEED_MAX'].astype(np.float32)
    speeds_chunk['N_VEHICLES'] = speeds_chunk['N_VEHICLES'].astype(np.int32)
    return speeds_chunk


def merge_speed_and_sensors(speed_chunk, sensors):
    # Merging speeds.train.csv.gz with sensors.csv.gz
    speeds_sensors = pd.merge(speed_chunk, sensors, on=['KEY', 'KM']).drop_duplicates().reset_index(
        drop=True)
    return speeds_sensors

def merge_SpeedSensors_and_events(speeds_chunk,events):
    # Merging speeds_sensors with events_train.csv.gz

    sensor_km = speeds_sensors.KM.values

    event_km_start = events.KM_START.values
    event_km_end = events.KM_END.values

    speed_sensors_key = speeds_sensors.KEY.values
    events_key = events.KEY_EVENTS.values

    speeds_sensors_time = speeds_sensors.DATETIME_UTC.values
    events_time_start = events.START_DATETIME_UTC.values
    events_time_end = events.END_DATETIME_UTC.values

    # The conditions for the merge
    i, j = np.where(
        (speed_sensors_key[:, None] == events_key) & (
                ((sensor_km[:, None] >= event_km_start) & (sensor_km[:, None] <= event_km_end)) | (
                (sensor_km[:, None] <= event_km_start) & (sensor_km[:, None] >= event_km_end))) & (
                speeds_sensors_time[:, None] >= events_time_start) & (speeds_sensors_time[:, None] <= events_time_end))

    # Performing the merge based on the above conditions
    speeds_sensors_events = pd.DataFrame(
        np.column_stack([speeds_sensors.values[i], events.values[j]]),
        columns=speeds_sensors.columns.append(events.columns)
    ).append(
        speeds_sensors[~np.in1d(np.arange(len(speeds_sensors)), np.unique(i))],
        ignore_index=True, sort=False
    )
    return speeds_sensors_events



chunk_list = []  # append each chunk df here

for chunk in speed_chunks:

    chunk = down_casting_data_types(chunk)
    chunk['DATETIME_UTC'] = pd.to_datetime(chunk['DATETIME_UTC'])

    # perform merging
    speeds_sensors = merge_speed_and_sensors(chunk,sensors)
    # perform merging
    chunk_processed = merge_SpeedSensors_and_events(speeds_sensors,events)

    # Once the data processing is done, append the chunk to list
    chunk_list.append(chunk_processed)

# concat the list into dataframe
dataset = pd.concat(chunk_list)